In [17]:
## Import packages ..... 
import cdsapi
import os
import os.path
from netCDF4 import Dataset
#import shutil # to move folders


## ..... and function
from FunctionsAndClass import year_copernicus
from FunctionsAndClass import path_length

In [2]:
## list all element necessary for download
class obs_elements:
    versions = ('v1.0','v2.0','v2020.0','v2020.0-v6.0-fg','v4.0','v4.03','v6.0')
    origins = ('berkearth','chirps','cmorph','cpc','cpc-conus','cru','gistemp','gpcc','imerg')
    # test only with chirps and cru at the beginning
    horizontal_aggregations = ('0_25_x_0_25','0_2_x_0_2','0_5_x_0_5','1_x_1','2_5_x_2_5','horizontal_average')
    time_aggregations = ('daily','monthly')
    regions = ('africa','global','conus','quasi_global')
    variables = ('precipitation','temperature','temperature_anomaly')
    statistics = ('NaN','maximum','mean','minimum') # NaN for when statistic is not needed

In [3]:
len(obs_elements.versions)

7

In [4]:
len(obs_elements.origins)

9

In [5]:
len(obs_elements.horizontal_aggregations)

6

In [6]:
len(obs_elements.time_aggregations)

2

In [7]:
len(obs_elements.regions)

4

In [8]:
len(obs_elements.variables)

3

In [9]:
len(obs_elements.statistics)

4

In [10]:
# create a list of iterable for the later for loop which will be use to download elements
list_obs_elements = []
for statistic in obs_elements.statistics:
    for origin in obs_elements.origins:
        for horizontal_aggregation in obs_elements.horizontal_aggregations:
            for time_aggregation in obs_elements.time_aggregations:
                for region in obs_elements.regions:
                    for variable in obs_elements.variables:
                        for version in obs_elements.versions:
                            list_obs_elements.append((version,origin,horizontal_aggregation,time_aggregation,region,variable,statistic))

In [11]:
list_obs_elements

[('v1.0',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'precipitation',
  'NaN'),
 ('v2.0',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'precipitation',
  'NaN'),
 ('v2020.0',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'precipitation',
  'NaN'),
 ('v2020.0-v6.0-fg',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'precipitation',
  'NaN'),
 ('v4.0',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'precipitation',
  'NaN'),
 ('v4.03',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'precipitation',
  'NaN'),
 ('v6.0',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'precipitation',
  'NaN'),
 ('v1.0', 'berkearth', '0_25_x_0_25', 'daily', 'africa', 'temperature', 'NaN'),
 ('v2.0', 'berkearth', '0_25_x_0_25', 'daily', 'africa', 'temperature', 'NaN'),
 ('v2020.0',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'africa',
  'temperature',
  'NaN'),
 ('v2020.0-v6.0-fg',
  'berkearth',
  '0_25_x_0_25',
  'daily',
  'afric

In [12]:
len(list_obs_elements)

36288

In [13]:
#def download_extract(path_for_file,source):
def download_extract(path_for_file):
    if not os.path.isdir(path_for_file): # path_for_file does not exists, need to ensure that is is created
        os.makedirs(path_for_file) # to ensure the creation of the path
    # unzip the downloaded file
    from zipfile import ZipFile
    zf = ZipFile('download.zip', 'r')
    zf.extractall(path_for_file) # if no precision of directory, extract in current directory
    zf.close()

    #if not os.path.isdir(file_download): # path_for_file does not exists, need to ensure that is is created
    #   os.makedirs(file_download) # to ensure the creation of the path
    # moving download to appropriate place
    #shutil.move('download.zip',file_download) # no need to delete 'download.zip' from inital place
    #shutil.move(source,path_for_file) # move extracted data to the path created for them
    #path_file=os.path.join(path_for_file,source)
    print('\n ------------------------------------- The downloaded file is extracted -------------------------------------')
    #os.remove('download.zip') # delete 'download.zip' from current directory
    return #path_file

In [14]:
# this function define the start_date, dependent on the origin
def f_start_date(origin):
    # start_date depend on the origin
    if origin == 'berkearth':
        start_date = 1950
    if origin == 'chirps':
        start_date = 1981
    if origin == 'cmorph':
        start_date = 1998
    if origin == 'cpc':
        start_date = 1979
    if origin == 'cpc-conus':
        start_date = 1950
    if origin == 'cru':
        start_date = 1950
    if origin == 'gistemp':
        start_date = 1950
    if origin == 'gpcc':
        start_date = 1950
    if origin == 'imerg':
        start_date = 2000
    return start_date

# this function defines the period of the set of data
def f_period_str(origin):
    # stop_date is the same for every origin
    stop_date = 2015
    start_date = f_start_date(origin) # start_date depend on the origin
    period = str(start_date) +' - '+str(stop_date)
    return period

## this function produce the year vector, which is dependent on the origin

def year_temp_pr_obs(origin):
    # stop_date is the same for every origin
    stop_date = 2015
    start_date = f_start_date(origin) # start_date depend on the origin
    # produce vector of years
    (year, year_str, index)= year_copernicus(start_date,stop_date)
    return year_str

In [15]:
## this function aim to download observation data of https://cds.climate.copernicus.eu/cdsapp#!/dataset/insitu-gridded-observations-global-and-regional?tab=form
# the function returns the path where the data has been downloaded
def try_download_copernicus_temp_pr_obs(version,origin,horizontal_aggregation,time_aggregation,region,variable,statistic,path_for_file,source):
    # produce the year vector, which is dependent on the origin
    year = year_temp_pr_obs(origin)
    
    # start the API request
    c = cdsapi.Client()# function to use the c.retrieve
    # basic needed dictionnary to give to the c.retrieve function the parameters asked by the user
    variables = {
                'format': 'zip', # this function is only designed to download and unzip zip files
                'version': version,
                'origin': origin,
                'year': year,
                'horizontal_aggregation': horizontal_aggregation,
                'time_aggregation': time_aggregation,
                'region': region,
                'variable': variable,
    }
    
    if statistic != 'NaN':
        variables['statistic'] = statistic
    
    try:
        c.retrieve(
            'insitu-gridded-observations-global-and-regional',
            variables,
            'download.zip') # the file in a zip format is registered in the current directory
    except:
        print('------------------------------- Some parameters are not matching -------------------------------')
        return # stop the function, because some data the user entered are not matching
    
    # function to extract the downloaded zip
    #path_file=download_extract(path_for_file,file_download,source)
    download_extract(path_for_file,source)
    os.remove('download.zip') # delete 'download.zip' from current directory
    return #path_file

In [20]:
Dataset(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\Observations\precipitation\africa\1981 - 2015\daily\0_25_x_0_25\chirps\v2.0\CHIRPS_total_precipitation_day_0.25x0.25_africa_1982_v2.0.nc')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    CDI: Climate Data Interface version 1.9.10 (https://mpimet.mpg.de/cdi)
    source: ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CHIRPS-2.0
    Conventions: CF-1.5
    GDAL: GDAL 2.0.2, released 2016/01/26
    title: CHIRPS-2.0 merged satellite / rain gauge precipitation estimate
    geospatial_lat_min: -40.005
    geospatial_lat_max: 40.005
    geospatial_lat_units: degrees_north
    geospatial_lon_min: -20.005
    geospatial_lon_max: 55.005
    geospatial_lon_units: degrees_east
    geospatial_lat_resolution: 0.25
    geospatial_lon_resolution: 0.25
    climexp_url: https://climexp.knmi.nl/select.cgi?chirps_20_25
    history: Fri Jun 18 07:50:10 2021: cdo -O -splityear /data/public/cdsdata/observations2/tmp/tmp2mopscww.nc /data/public/cdsdata/observations2/CHIRPS_total_precipitation_day_0.25x0.25_africa_
Fri Jun 18 07:50:10 2021: cdo -O -splityear /data/public/cdsdata/observations2/tmp/tmp

In [ ]:
# define path and source of data
out_path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\Observations'
source = 'copernicus-data-in-situ-and-satellite'
# start testing if data exists and if yes, download them with function try_download_copernicus_temp_pr_obs
count = 0
for (version, origin, horizontal_aggregation,time_aggregation, region,variable,statistic) in list_obs_elements:
    count +=1
    print('\n')    
    print('version: ' +version)
    print('\n')    
    print('origin: ' +origin)
    print('\n')
    print('horizontal_aggregation: ' +horizontal_aggregation)
    print('\n')
    print('time_aggregation: ' +time_aggregation)
    print('\n')
    print('region: ' +region )
    print('\n')
    print('variable: '+ variable)
    print('\n')
    print('statistic: ' +statistic)
    print('\n')
    period = f_period_str(origin) # produce string of the considered period, to integrate period in the path
    path_for_file = os.path.join(out_path,variable, region,period, time_aggregation, horizontal_aggregation, origin, version) # create string of the path
    if statistic != 'NaN': # a statitics is requested
        path_for_file = os.path.join(path_for_file, statistic) # add statistic in the path
    if os.path.isdir(path_for_file):
        # the path for the file already exists, so the files too
        continue # continue to the next loop
    path_for_file=path_length(path_for_file)# test if path is too long
    # try to download the data with the parameter of the loop. If the parameter match, the file will be dowloaded 
    # and extract within the same function try_download_copernicus_temp_pr_obs
    try_download_copernicus_temp_pr_obs(version,origin,horizontal_aggregation,time_aggregation,region,variable,statistic,path_for_file,source)
    if 'download.zip' in os.listdir():
        # there is a file named 'download.zip' in the current directory
        os.remove('download.zip') # delete 'download.zip' from current directory



version: v1.0


origin: berkearth


horizontal_aggregation: 0_25_x_0_25


time_aggregation: daily


region: africa


variable: precipitation


statistic: NaN




2023-05-25 09:14:05,256 INFO Welcome to the CDS
2023-05-25 09:14:05,256 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/insitu-gridded-observations-global-and-regional
2023-05-25 09:14:05,307 INFO Request is queued
2023-05-25 09:14:06,348 INFO Request is failed
2023-05-25 09:14:06,348 ERROR Message: the request you have submitted is not valid
2023-05-25 09:14:06,356 ERROR Reason:  There is no data matching your request. Check that you have specified the correct fields and values.
2023-05-25 09:14:06,356 ERROR   Traceback (most recent call last):
2023-05-25 09:14:06,361 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163, in _consume
2023-05-25 09:14:06,362 ERROR       result = handle_locally()
2023-05-25 09:14:06,364 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 252, in <lambda>
2023-05-25 09:14:06,366 ERROR       lambda: self.handle_exception(context, e),
2023-05-25 09:14:06,367 ERROR     File "/o

------------------------------- Some parameters are not matching -------------------------------


version: v2.0


origin: berkearth


horizontal_aggregation: 0_25_x_0_25


time_aggregation: daily


region: africa


variable: precipitation


statistic: NaN




2023-05-25 09:14:07,665 INFO Request is failed
2023-05-25 09:14:07,665 ERROR Message: the request you have submitted is not valid
2023-05-25 09:14:07,669 ERROR Reason:  There is no data matching your request. Check that you have specified the correct fields and values.
2023-05-25 09:14:07,670 ERROR   Traceback (most recent call last):
2023-05-25 09:14:07,673 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163, in _consume
2023-05-25 09:14:07,674 ERROR       result = handle_locally()
2023-05-25 09:14:07,674 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 252, in <lambda>
2023-05-25 09:14:07,677 ERROR       lambda: self.handle_exception(context, e),
2023-05-25 09:14:07,678 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 383, in handle_exception
2023-05-25 09:14:07,678 ERROR       raise exception
2023-05-25 09:14:07,686 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163

------------------------------- Some parameters are not matching -------------------------------


version: v2020.0


origin: berkearth


horizontal_aggregation: 0_25_x_0_25


time_aggregation: daily


region: africa


variable: precipitation


statistic: NaN




...

2023-05-25 09:24:32,191 INFO Request is failed
2023-05-25 09:24:32,191 ERROR Message: the request you have submitted is not valid
2023-05-25 09:24:32,191 ERROR Reason:  There is no data matching your request. Check that you have specified the correct fields and values.
2023-05-25 09:24:32,191 ERROR   Traceback (most recent call last):
2023-05-25 09:24:32,199 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163, in _consume
2023-05-25 09:24:32,199 ERROR       result = handle_locally()
2023-05-25 09:24:32,205 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 252, in <lambda>
2023-05-25 09:24:32,206 ERROR       lambda: self.handle_exception(context, e),
2023-05-25 09:24:32,206 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 383, in handle_exception
2023-05-25 09:24:32,206 ERROR       raise exception
2023-05-25 09:24:32,211 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163

------------------------------- Some parameters are not matching -------------------------------


version: v2.0


origin: berkearth


horizontal_aggregation: 0_2_x_0_2


time_aggregation: monthly


region: quasi_global


variable: temperature


statistic: NaN




2023-05-25 09:24:34,971 INFO Request is failed
2023-05-25 09:24:34,971 ERROR Message: the request you have submitted is not valid
2023-05-25 09:24:34,971 ERROR Reason:  There is no data matching your request. Check that you have specified the correct fields and values.
2023-05-25 09:24:34,971 ERROR   Traceback (most recent call last):
2023-05-25 09:24:34,979 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163, in _consume
2023-05-25 09:24:34,979 ERROR       result = handle_locally()
2023-05-25 09:24:34,979 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 252, in <lambda>
2023-05-25 09:24:34,979 ERROR       lambda: self.handle_exception(context, e),
2023-05-25 09:24:34,987 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 383, in handle_exception
2023-05-25 09:24:34,987 ERROR       raise exception
2023-05-25 09:24:34,987 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163

------------------------------- Some parameters are not matching -------------------------------


version: v2020.0


origin: berkearth


horizontal_aggregation: 0_2_x_0_2


time_aggregation: monthly


region: quasi_global


variable: temperature


statistic: NaN




2023-05-25 09:24:37,820 INFO Request is failed
2023-05-25 09:24:37,820 ERROR Message: the request you have submitted is not valid
2023-05-25 09:24:37,820 ERROR Reason:  There is no data matching your request. Check that you have specified the correct fields and values.
2023-05-25 09:24:37,820 ERROR   Traceback (most recent call last):
2023-05-25 09:24:37,828 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163, in _consume
2023-05-25 09:24:37,828 ERROR       result = handle_locally()
2023-05-25 09:24:37,828 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 252, in <lambda>
2023-05-25 09:24:37,828 ERROR       lambda: self.handle_exception(context, e),
2023-05-25 09:24:37,828 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 383, in handle_exception
2023-05-25 09:24:37,828 ERROR       raise exception
2023-05-25 09:24:37,837 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163

In [ ]:
count

In [ ]:
## test download of chirps

period = f_period_str('chirps') # produce string of the considered period, to integrate period in the path
horizontal_aggregation = '0_25_x_0_25'
time_aggregation = 'daily'
region = 'africa'
variable = 'precipitation'
version = 'v2.0'
origin = 'chirps'

out_path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\Observations'
path_for_file = os.path.join(out_path,variable, region,period, time_aggregation, horizontal_aggregation, origin, version) # create string of the path

c = cdsapi.Client()

c.retrieve(
    'insitu-gridded-observations-global-and-regional',
    {
        'format': 'zip',
        'version': 'v2.0',
        'origin': 'chirps',
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020', '2021',
        ],
        'horizontal_aggregation': '0_25_x_0_25',
        'time_aggregation': 'daily',
        'region': 'africa',
        'variable': 'precipitation',
    },
    'download.zip')

download_extract(path_for_file)
